In [1]:
import json
import numpy as np
import ipyvolume as ipv
import ipyvolume.pylab as p3
import matplotlib.cm as cm
import sys
import matplotlib as plt
from ipywidgets import interact, fixed,interact_manual,IntSlider,VBox


In [2]:
# Load Data
try:
#     filename = sys.argv[1]
    filename = "230718_135850"
except:
    print('Provide prefix of data you want to animate in format yymmdd_hhmmss as command line argument, e.g.:\n >python animation.py 201005_111211')
#     sys.exit()
try:
    data = np.loadtxt('./logfiles/{}_data.txt'.format(filename), delimiter=',')
    with open('./logfiles/{}_meta.txt'.format(filename), 'r') as f:
        meta = json.loads(f.read())
except:
    print('Data file with prefix {} does not exist.\nProvide prefix of data you want to animate in format yymmdd_hhmmss as command line argument, e.g.:\n >python animation.py 201005_111211'.format(filename))
#     sys.exit()

In [3]:
# Read Experimental Parameters
clock_freq = meta['Clock frequency [Hz]']
clock_rate = 1000/clock_freq # [ms]
arena = meta['Arena [mm]']
timesteps = data.shape[0]
fishes = int(data.shape[1]/8)

# Format Data
x = data[:, :1]
y = data[:, 1:2]
z = data[:, 2:3]
phi = data[:, 3:4]
#vx = data[:, 4:5]
#vy = data[:, 5:6]
#vz = data[:, 6:7]

for ii in range(1,fishes):
    x = np.concatenate((x, data[:, 4*ii:4*ii+1]), axis=1)
    y = np.concatenate((y, data[:, 4*ii+1:4*ii+2]), axis=1)
    z = np.concatenate((z, data[:, 4*ii+2:4*ii+3]), axis=1)
    phi = np.concatenate((phi, data[:, 4*ii+3:4*ii+4]), axis=1)
    #vx = np.concatenate((vx, data[:, 4*(fishes+ii):4*(fishes+ii)+1]), axis=1)
    #vy = np.concatenate((vy, data[:, 4*(fishes+ii)+1:4*(fishes+ii)+2]), axis=1)
    #vz = np.concatenate((vz, data[:, 4*(fishes+ii)+2:4*(fishes+ii)+3]), axis=1)

# Colors
v = np.sqrt(x**2 + y**2 + z**2)
v -= v.min(); v /= v.max()
colors = np.array([cm.Blues(k) for k in v])

In [18]:
# Create Animation
f = p3.figure()
f.animation = 0
p3.xlim(0, arena[0])
p3.ylim(0, arena[1])
p3.zlim(0, arena[2])
p3.style.use('dark')

time=0
quiver = p3.quiver(x[time, :], y[time, :], z[time, :], np.cos(phi[time, :]), np.sin(phi[time, :]), np.zeros((1,len(phi[time, :]))),size=6, color=colors[time,:,:3])
# quiver = p3.quiver(x, y, z, np.cos(phi), np.sin(phi), np.zeros((1,len(phi))),size=6, color=colors[:,:,:3])


p3.animation_control(quiver, interval=clock_rate)

In [19]:
p3.show()

In [20]:
def set_angles(fig, i, fraction):
#     print(i)
    quiver.x = x[i, :]
    quiver.y = y[i, :]
    quiver.z = z[i, :]
    quiver.vx = np.cos(phi[i, :])
    quiver.vy = np.sin(phi[i, :])
    quiver.vz = np.zeros((1,len(phi[i, :])))
    quiver.color = colors[i, :, :3]


frames=200
p3.movie('./animations/{}_animation.mp4'.format(filename), function=set_angles, frames=frames)

Output()

In [16]:
print(colors.shape)

(201, 20, 4)
